# COVID-19 Hospitalization, ICU & Mortality Dashboard Update
Center for Human Dynamics in the Mobile Age (HDMA) at San Diego State University

Jessica Embury

### Modules

In [23]:
import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from arcgis.mapping import WebMap
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import json
import sys
import webbrowser

### User Entered Variables

In [24]:
###############
###SET DATES###
###############
file_date = '5SEP2020'

##########################
###SET PATHS IN AND OUT###
##########################
#PATHS IN
sra_in = 'data/sra_info.csv'
hosp_in = 'COVID19_HospRate_{}.csv'.format(file_date)
icu_in = 'COVID19_ICURate_{}.csv'.format(file_date)
mort_in = 'COVID19_MortalityRate_{}.csv'.format(file_date)

#PATHS OUT
hosp_out = 'covid19_hosp_upload.csv'
icu_out = 'covid_icu_upload.csv'
mort_out = 'covid_mort_upload.csv'

#Cumulative Dates Files
hosp_dates = 'covid19_hospitalization.csv'
icu_dates = 'covid_icu.csv'
mort_dates = 'covid_mortality.csv'

######################################
###ARCGIS ACCOUNT LOGIN INFORMATION###
######################################
#reference for authentication schemes: https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/
pro = "pro"
#portal =''
#username=''
#password=''

################################
###ARCGIS DETAILS FOR UPDATES###
################################
hosp_layer_id = '2f3443de1a9345d79316484d6c273342'
hosp_map_id = '3b5af1a898ae49e3825bac21badea77c'
hosp_csv_id = '0413f1006e94418b8ecd3af609ec5dff'

icu_layer_id ='164de458148f4c9db3708cc8e99ddbc8'
icu_map_id = '128d6eb1547c4958818a47464da1a079'
icu_csv_id = '7dc2c321393e402b9931d5595fae8678'

mort_layer_id = '1bf213866fad441db609577cad03c7d9'
mort_map_id = '64f43b6d62ef46c18bf6db55fce5afce'
mort_csv_id = '11047e66ca6f483d8873078ef392e18c'

dash = 'https://arcg.is/0amqvz'

### Connect to ArcGIS Account

In [25]:
gis = GIS(pro)
#gis = GIS(portal, username, password)

### Functions for Map Modifications

In [26]:
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''    
    m = gis.content.get(map_id)
    data = m.get_data()
    print(m)    
    return data

def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)  
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}
    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)    
    return update

# HOSPITALIZATION

### Format Data

In [27]:
#BASE SRA DATA
sra = pd.read_csv(sra_in)
print(len(sra))
sra.head()

41


,sra_num,sra_name,latitude,longitude
0,1,Central San Diego,32.722644,-117.141073
1,2,Peninsula,32.742274,-117.216035
2,3,Coronado,32.657317,-117.143062
3,4,National City,32.665847,-117.099737
4,5,Southeastern San Diego,32.685705,-117.038621


In [28]:
hosp = pd.read_csv(hosp_in)
date = hosp['Date'][0]
print(date)
hosp.head()

9/5/2020


,Geography,Allocated\nHospitalization\nCount (Raw),COVID-19\nTotal\nHospitalization\nRate\n(Suppressed),Date
0,Alpine,5.5,34.3,9/5/2020
1,Anza-Borrego Springs,NaN,NaN,9/5/2020
2,Carlsbad,38.2,30.2,9/5/2020
3,Central Region,694.0,134.7,9/5/2020
4,Central San Diego,207.7,98.5,9/5/2020


In [29]:
del hosp['COVID-19\nTotal\nHospitalization\nRate\n(Suppressed)']
hosp = hosp.rename(columns = {'Allocated\nHospitalization\nCount (Raw)':'hosp_count', 'Geography':'sra_name'})
hosp = hosp.fillna(0)
hosp.head()

,sra_name,hosp_count,Date
0,Alpine,5.5,9/5/2020
1,Anza-Borrego Springs,0.0,9/5/2020
2,Carlsbad,38.2,9/5/2020
3,Central Region,694.0,9/5/2020
4,Central San Diego,207.7,9/5/2020


In [30]:
#merge to sra df with coordinates
hosp_merge = sra.merge(hosp, how='left', on='sra_name')
hosp_merge['label'] = hosp_merge['hosp_count'].astype(str)
print(len(hosp_merge))
hosp_merge.head()

41


,sra_num,sra_name,latitude,longitude,hosp_count,Date,label
0,1,Central San Diego,32.722644,-117.141073,207.7,9/5/2020,207.7
1,2,Peninsula,32.742274,-117.216035,18.7,9/5/2020,18.7
2,3,Coronado,32.657317,-117.143062,8.0,9/5/2020,8.0
3,4,National City,32.665847,-117.099737,150.6,9/5/2020,150.6
4,5,Southeastern San Diego,32.685705,-117.038621,304.7,9/5/2020,304.7


In [31]:
#export csv for feature layer overwrite
hosp_merge.to_csv(hosp_out, index=False)

In [32]:
#save csv with a column for every date
#prepare new data for addition to existing csv
hosp_temp = hosp_merge[['sra_name','hosp_count']]
hosp_temp = hosp_temp.rename(columns = {'hosp_count':'{}'.format(date)})
hosp_temp.head()

,sra_name,9/5/2020
0,Central San Diego,207.7
1,Peninsula,18.7
2,Coronado,8.0
3,National City,150.6
4,Southeastern San Diego,304.7


In [33]:
#add new date column to existing csv
hosp_cum = pd.read_csv(hosp_dates)
hosp_cum = hosp_cum.merge(hosp_temp, on='sra_name')
hosp_cum.to_csv(hosp_dates, index=False)
hosp_cum.head()

,sra_num,sra_name,latitude,longitude,6/3/2020,6/10/2020,6/17/2020,6/24/2020,7/1/2020,7/8/2020,7/14/2020,7/28/2020,8/4/2020,8/11/2020,8/18/2020,8/24/2020,8/29/2020,9/5/2020
0,1,Central San Diego,32.722644,-117.141073,94.39,102.22,109.7,114.9,127.6,138.8,150.6,176.5,189.0,195.0,198.1,199.9,205.7,207.7
1,2,Peninsula,32.742274,-117.216035,7.67,7.67,9.7,9.7,10.6,10.6,12.3,14.8,14.8,16.2,16.7,17.7,17.7,18.7
2,3,Coronado,32.657317,-117.143062,0.00,0.00,0.0,5.0,5.0,5.0,5.0,6.0,7.0,8.0,8.0,8.0,8.0,8.0
3,4,National City,32.665847,-117.099737,81.06,82.99,88.8,97.5,103.3,110.0,117.7,124.6,131.3,137.1,139.0,141.9,145.8,150.6
4,5,Southeastern San Diego,32.685705,-117.038621,118.17,125.60,141.4,150.3,168.2,184.6,200.2,245.2,263.0,274.8,284.4,289.9,294.9,304.7


### Overwrite Hospitalization Feature Layer and CSV File

In [34]:
#overwrite master csv with all dates data
csv_item = gis.content.get(hosp_csv_id)
csv_item.update({}, hosp_dates)

True

In [35]:
#get feature layer containing updated data 
hosp_layer = gis.content.get(hosp_layer_id)
hosp_layer

hosp_layer_collection = FeatureLayerCollection.fromitem(hosp_layer)

#call the overwrite() method which can be accessed using the manager property
hosp_layer_collection.manager.overwrite(hosp_out)

{'success': True}

### Modify Hospitalization Map Symbology

In [36]:
#get hospitalization max for graduated point symbology
hosp_max = hosp_merge['hosp_count'].max()

#get map data
hosp_map = get_map(hosp_map_id)

#set max value for graduated points symbols
hosp_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = hosp_max.item()
hosp_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = hosp_max.item()
hosp_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = hosp_max.item()
hosp_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = hosp_max.item()

#update map to save changes
hosp_map_update = update_map(hosp_map_id, hosp_map)
hosp_map_update

<Item title:"COVID-19 Hospitalizations by San Diego SRA" type:Web Map owner:jembury8568_SDSUGeo>


True

# ICU BEDS

### Format Data

In [37]:
icu = pd.read_csv(icu_in)
date = icu['Date'][0]
icu.head()

,Geography,Allocated\nICU\nCount\n(Raw),COVID-19\nTotal ICU\nRate\n(Suppressed),Date
0,Alpine,NaN,NaN,9/5/2020
1,Anza-Borrego Springs,NaN,NaN,9/5/2020
2,Carlsbad,NaN,NaN,9/5/2020
3,Central Region,170.0,33.0,9/5/2020
4,Central San Diego,54.6,25.9,9/5/2020


In [38]:
del icu['COVID-19\nTotal ICU\nRate\n(Suppressed)']
icu = icu.rename(columns = {'Allocated\nICU\nCount\n(Raw)':'icu_count', 'Geography':'sra_name'})
icu = icu.fillna(0)
icu.head()

,sra_name,icu_count,Date
0,Alpine,0.0,9/5/2020
1,Anza-Borrego Springs,0.0,9/5/2020
2,Carlsbad,0.0,9/5/2020
3,Central Region,170.0,9/5/2020
4,Central San Diego,54.6,9/5/2020


In [39]:
#merge to sra df with coordinates
icu_merge = sra.merge(icu, how='left', on='sra_name')
icu_merge['label'] = icu_merge['icu_count'].astype(str)
print(len(icu_merge))
icu_merge.head()

41


,sra_num,sra_name,latitude,longitude,icu_count,Date,label
0,1,Central San Diego,32.722644,-117.141073,54.6,9/5/2020,54.6
1,2,Peninsula,32.742274,-117.216035,0.0,9/5/2020,0.0
2,3,Coronado,32.657317,-117.143062,0.0,9/5/2020,0.0
3,4,National City,32.665847,-117.099737,30.9,9/5/2020,30.9
4,5,Southeastern San Diego,32.685705,-117.038621,78.5,9/5/2020,78.5


In [40]:
#export csv for feature layer overwrite
icu_merge.to_csv(icu_out, index=False)

In [41]:
#save csv with a column for every date
#prepare new data for addition to existing csv
icu_temp = icu_merge[['sra_name','icu_count']]
icu_temp = icu_temp.rename(columns = {'icu_count':'{}'.format(date)})
icu_temp.head()

,sra_name,9/5/2020
0,Central San Diego,54.6
1,Peninsula,0.0
2,Coronado,0.0
3,National City,30.9
4,Southeastern San Diego,78.5


In [42]:
#add new date column to existing csv
icu_cum = pd.read_csv(icu_dates)
icu_cum = icu_cum.merge(icu_temp, on='sra_name')
icu_cum.to_csv(icu_dates, index=False)
icu_cum.head()

,sra_num,sra_name,latitude,longitude,6/4/2020,6/10/2020,6/17/2020,6/24/2020,7/1/2020,7/8/2020,7/14/2020,7/28/2020,8/4/2020,8/11/2020,8/18/2020,8/24/2020,8/29/2020,9/5/2020
0,1,Central San Diego,32.722644,-117.141073,24.467,27.401,29.6,32.7,35.9,37.7,39.3,50.9,50.9,51.8,53.6,53.2,53.2,54.6
1,2,Peninsula,32.742274,-117.216035,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Coronado,32.657317,-117.143062,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,National City,32.665847,-117.099737,17.371,17.372,19.3,21.2,23.2,25.1,25.1,27.0,27.0,29.0,29.0,29.0,29.9,30.9
4,5,Southeastern San Diego,32.685705,-117.038621,32.928,37.816,42.7,44.6,51.2,55.2,58.3,69.1,69.1,75.1,74.9,75.5,76.7,78.5


### Overwrite ICU Feature Layer and CSV File

In [43]:
#overwrite master csv with all dates data
csv_item = gis.content.get(icu_csv_id)
csv_item.update({}, icu_dates)

True

In [44]:
#get feature layer containing updated data 
icu_layer = gis.content.get(icu_layer_id)
icu_layer

icu_layer_collection = FeatureLayerCollection.fromitem(icu_layer)

#call the overwrite() method which can be accessed using the manager property
icu_layer_collection.manager.overwrite(icu_out)

{'success': True}

### Modify ICU Map Symbology

In [45]:
#get icu max for graduated point symbology
icu_max = icu_merge['icu_count'].max()

#get map data
icu_map = get_map(icu_map_id)

#set max value for graduated points symbols
icu_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = icu_max.item()
icu_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = icu_max.item()
icu_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = icu_max.item()
icu_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = icu_max.item()

#update map to save changes
icu_map_update = update_map(icu_map_id, icu_map)
icu_map_update

<Item title:"COVID-19 ICU Beds by San Diego SRA" type:Web Map owner:jembury8568_SDSUGeo>


True

# MORTALITY

### Format Data

In [46]:
mort = pd.read_csv(mort_in)
date = mort['Date'][0]
print(date)

mort.head()

9/5/2020


,Geography,Allocated\nDeath\nCount\n(Raw),COVID-19\nTotal\nMortality\nRate\n(Suppressed),Date
0,Alpine,NaN,NaN,9/5/2020
1,Anza-Borrego Springs,NaN,NaN,9/5/2020
2,Carlsbad,NaN,NaN,9/5/2020
3,Central Region,148.0,28.7,9/5/2020
4,Central San Diego,42.3,20.1,9/5/2020


In [47]:
del mort['COVID-19\nTotal\nMortality\nRate\n(Suppressed)']
mort = mort.rename(columns = {'Allocated\nDeath\nCount\n(Raw)':'mort_count', 'Geography':'sra_name'})
mort = mort.fillna(0)
mort.head()

,sra_name,mort_count,Date
0,Alpine,0.0,9/5/2020
1,Anza-Borrego Springs,0.0,9/5/2020
2,Carlsbad,0.0,9/5/2020
3,Central Region,148.0,9/5/2020
4,Central San Diego,42.3,9/5/2020


In [48]:
#merge to sra df with coordinates
mort_merge = sra.merge(mort, how='left', on='sra_name')
mort_merge['label'] = mort_merge['mort_count'].astype(str)
print(len(mort_merge))
mort_merge.head()

41


,sra_num,sra_name,latitude,longitude,mort_count,Date,label
0,1,Central San Diego,32.722644,-117.141073,42.3,9/5/2020,42.3
1,2,Peninsula,32.742274,-117.216035,5.7,9/5/2020,5.7
2,3,Coronado,32.657317,-117.143062,0.0,9/5/2020,0.0
3,4,National City,32.665847,-117.099737,46.3,9/5/2020,46.3
4,5,Southeastern San Diego,32.685705,-117.038621,77.2,9/5/2020,77.2


In [49]:
#export csv for feature layer overwrite
mort_merge.to_csv(mort_out, index=False)

In [50]:
#save csv with a column for every date
#prepare new data for addition to existing csv
mort_temp = mort_merge[['sra_name','mort_count']]
mort_temp = mort_temp.rename(columns = {'mort_count':'{}'.format(date)})
mort_temp.head()

,sra_name,9/5/2020
0,Central San Diego,42.3
1,Peninsula,5.7
2,Coronado,0.0
3,National City,46.3
4,Southeastern San Diego,77.2


In [51]:
#add new date column to existing csv
mort_cum = pd.read_csv(mort_dates)
mort_cum = mort_cum.merge(mort_temp, on='sra_name')
mort_cum.to_csv(mort_dates, index=False)
mort_cum.head()

,sra_num,sra_name,latitude,longitude,6/3/20,6/10/2020,6/17/2020,6/24/2020,7/1/2020,7/8/2020,7/14/2020,7/28/2020,8/4/2020,8/11/2020,8/18/2020,8/24/2020,8/29/2020,9/5/2020
0,1,Central San Diego,32.722644,-117.141073,15.584,16.985,19.1,21.8,23.5,25.6,28.8,35.8,38.5,40.5,40.9,41.3,41.3,42.3
1,2,Peninsula,32.742274,-117.216035,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,5.2,5.2,5.2,5.2,5.7,5.7
2,3,Coronado,32.657317,-117.143062,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,National City,32.665847,-117.099737,21.224,23.154,24.1,25.1,26.0,32.8,34.7,38.6,41.5,41.5,43.4,43.4,43.4,46.3
4,5,Southeastern San Diego,32.685705,-117.038621,16.742,17.457,19.4,20.3,24.4,32.1,41.5,63.4,64.7,66.8,70.3,73.0,73.3,77.2


### Overwrite Mortality Feature Layer and CSV File

In [52]:
#overwrite master csv with all dates data
csv_item = gis.content.get(mort_csv_id)
csv_item.update({}, mort_dates)

True

In [53]:
#get feature layer containing updated data 
mort_layer = gis.content.get(mort_layer_id)
mort_layer

mort_layer_collection = FeatureLayerCollection.fromitem(mort_layer)

#call the overwrite() method which can be accessed using the manager property
mort_layer_collection.manager.overwrite(mort_out)

{'success': True}

### Modify Mortality Map Symbology

In [54]:
#get mortality max for graduated point symbology
mort_max = mort_merge['mort_count'].max()

#get map data
mort_map = get_map(mort_map_id)

#set max value for graduated points symbols
mort_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = mort_max.item()
mort_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = mort_max.item()
mort_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = mort_max.item()
mort_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = mort_max.item()

#update map to save changes
mort_map_update = update_map(mort_map_id, mort_map)
mort_map_update

<Item title:"COVID-19 Mortality by San Diego SRA" type:Web Map owner:jembury8568_SDSUGeo>


True

In [55]:
webbrowser.open(dash, new=2)

True